# Build a cuisine recommender

In [11]:
import pandas as pd
data = pd.read_csv('../data/cleaned_cuisines.csv')
X = data.iloc[:,2:]   # Features
y = data[['cuisine']] # Labels

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score,precision_score,confusion_matrix,classification_report

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)
model = SVC(kernel='linear', C=10, probability=True,random_state=0)
model.fit(X_train,y_train.values.ravel())

y_pred = model.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

     chinese       0.65      0.74      0.69       224
      indian       0.90      0.89      0.90       256
    japanese       0.81      0.78      0.80       237
      korean       0.89      0.75      0.81       236
        thai       0.78      0.82      0.80       246

    accuracy                           0.80      1199
   macro avg       0.80      0.80      0.80      1199
weighted avg       0.81      0.80      0.80      1199



Good model! Now lets convert it to Onnx, which will allow us to use it in a web app.<br>Note, we have used a tensor number of 380 - this corresponds to the number of ingredients in the dataset. In ML, input and output data are typically represented as tensors - the tensor number is often the dimensionality of those tensors. 

In [14]:
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

initial_type = [('float_input', FloatTensorType([None, 380]))]
options = {id(model): {'nocl': True, 'zipmap': False}} # Nocl means no class information is included - reduces model size

In [15]:
onx = convert_sklearn(model, initial_types=initial_type, options=options)
with open("./model.onnx", "wb") as f:
    f.write(onx.SerializeToString())

Recommend opening this ONNX model in Netron, to visually verify it has created correctly.

Now, this model can be placed in a web app, and accessed offline if needed (it's now a static model)